<a href="https://colab.research.google.com/github/RICHAR-SL/IA/blob/main/resumen_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Forzar reinstalación limpia
!pip uninstall -y transformers datasets accelerate torch torchaudio torchvision
!pip install --no-cache-dir --upgrade pip
!pip install --no-cache-dir --upgrade \
  torch torchvision torchaudio \
  transformers[torch] datasets[audio] accelerate torchcodec
!pip install git+https://github.com/huggingface/transformers



para resumir texto

In [ ]:
# Forzar reinstalación limpia
!pip uninstall -y transformers datasets accelerate torch torchaudio torchvision
!pip install --no-cache-dir --upgrade pip
!pip install --no-cache-dir --upgrade \
  torch torchvision torchaudio \
  transformers[torch] datasets[audio] accelerate torchcodec
!pip install git+https://github.com/huggingface/transformers
!pip install easyocr
!pip install gtts
!pip install pyphen


Found existing installation: transformers 4.56.0.dev0
Uninstalling transformers-4.56.0.dev0:
  Successfully uninstalled transformers-4.56.0.dev0
Found existing installation: datasets 4.0.0
Uninstalling datasets-4.0.0:
  Successfully uninstalled datasets-4.0.0
Found existing installation: accelerate 1.10.0
Uninstalling accelerate-1.10.0:
  Successfully uninstalled accelerate-1.10.0
Found existing installation: torch 2.8.0
Uninstalling torch-2.8.0:
  Successfully uninstalled torch-2.8.0
Found existing installation: torchaudio 2.8.0
Uninstalling torchaudio-2.8.0:
  Successfully uninstalled torchaudio-2.8.0
Found existing installation: torchvision 0.23.0
Uninstalling torchvision-0.23.0:
  Successfully uninstalled torchvision-0.23.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.1/888.1 MB 239.9 MB/s  0:00:03
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 196.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 254.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

article = """Entiendo que no puedo suplicarle una vez más
Pero nada se detiene
Solo vivo para ti
Dame solo un beso
Que me alcance hasta morir
Como un vicio que me duele
Quiero mirarte a los ojos
Luna, no me abandones más
Que tiendo a recuperarme
En la cuna de tus cráteres
Silencio, se abre la tierra
Y se alzan los mares
Al compás del volcán
♪
Cuando te me acercas se acelera mi motor
Me da fiebre, me hago fuego
Y me vuelvo a consumir
Dame solo un beso
Que me alcance hasta morir
Como un vicio que me duele
Quiero mirarte a los ojos
♪
Luna, no me abandones más
Que tiendo a recuperarme
En la cuna de tus cráteres
Silencio, se abre la tierra
Y se alzan los mares
Al compás del volcán
"""
summary = summarizer(article, max_length=130, min_length=30)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [2]:
print(summary[0]['summary_text'])
print(f"Número de caracteres: {len(article)}")
print(f"Número de palabras: {len(article.split())}")

Pero nada se detiene. Entiendo que no puedo suplicarle una vez más. Quiero mirarte a los ojos                Luna, no me abandones más                Que tiendo a recuperarme.
Número de caracteres: 672
Número de palabras: 135


In [3]:
text = summary[0]['summary_text']
print(f"Resumen: {text}")
print(f"Número de caracteres: {len(text)}")
print(f"Número de palabras: {len(text.split())}")

Resumen: Pero nada se detiene. Entiendo que no puedo suplicarle una vez más. Quiero mirarte a los ojos                Luna, no me abandones más                Que tiendo a recuperarme.
Número de caracteres: 175
Número de palabras: 26


audio


In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset

# Selección de dispositivo
device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

# Modelo
model_id = "openai/whisper-large-v3"

# Cargar modelo y procesador
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id,
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=True,
    use_safetensors=True,
).to(device)

processor = AutoProcessor.from_pretrained(model_id)

# Crear pipeline con timestamps
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=0 if torch.cuda.is_available() else -1,
    return_timestamps=True,  # Esto activa el modo long-form
)

# Transcribir archivo MP3 largo
audio_path = "/content/Luna - Zoé (Letra_Lyrics) (Unplugged)(MP3_160K).mp3"
result = pipe(audio_path)

# Imprimir resultado
print("🎧 Transcripción:")
if isinstance(result, dict):
    print(result["text"])
else:
    for chunk in result:
        print(f"[{chunk['timestamp'][0]:.2f} - {chunk['timestamp'][1]:.2f}] {chunk['text']}")


Device set to use cuda:0
Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.


In [ ]:

import easyocr
from transformers import pipeline
from gtts import gTTS
from IPython.display import display, Audio, Markdown

# OCR
reader = easyocr.Reader(['es'])
result = reader.readtext('/content/20250815_113804.jpg')
texto = " ".join([res[1] for res in result])
print("Texto extraído:")
print(texto)

# Corrección
corrector = pipeline('text2text-generation', model='t5-base')
texto_corregido = corrector(f"corregir: {texto}")[0]['generated_text']
print("\nTexto corregido:")
print(texto_corregido)

# Paráfrasis
parafraseador = pipeline("text2text-generation", model="Vamsi/T5_Paraphrase_Paws")
texto_parafraseado = parafraseador(texto_corregido)[0]['generated_text']

# Mostrar texto parafraseado y audio
display(Markdown(f"### Texto parafraseado:\n\n{texto_parafraseado}"))

tts = gTTS(text=texto_parafraseado, lang='es')
tts.save('salida_audio.mp3')
display(Audio('salida_audio.mp3', autoplay=False))


In [ ]:
import cv2
from PIL import Image
import numpy as np

# Leer imagen
img_path = '/content/20250815_113804.jpg'
img = cv2.imread(img_path)

# Convertir a escala de grises
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Aplicar umbral adaptativo
thresh = cv2.adaptiveThreshold(
    gray, 255,
    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    cv2.THRESH_BINARY,
    11, 2
)

# Guardar la imagen procesada
cv2.imwrite('/content/preprocesada.jpg', thresh)
Image.open('/content/preprocesada.jpg').show()


In [ ]:
import easyocr
reader = easyocr.Reader(['es'])
result = reader.readtext('/content/preprocesada.jpg')
texto = " ".join([res[1] for res in result])
print("Texto extraído:", texto)
# Corrección
corrector = pipeline('text2text-generation', model='t5-base')
texto_corregido = corrector(f"corregir: {texto}")[0]['generated_text']
print("\nTexto corregido:")
print(texto_corregido)

# Paráfrasis
parafraseador = pipeline("text2text-generation", model="Vamsi/T5_Paraphrase_Paws")
texto_parafraseado = parafraseador(texto_corregido)[0]['generated_text']

# Mostrar texto parafraseado y audio
display(Markdown(f"### Texto parafraseado:\n\n{texto_parafraseado}"))

tts = gTTS(text=texto_parafraseado, lang='es')
tts.save('salida_audio.mp3')
display(Audio('salida_audio.mp3', autoplay=False))

In [10]:
from IPython.display import IFrame
IFrame(src="https://TU-NGROK-URL", width=800, height=600)